In [28]:
import csv
import opencv
import sklearn
import numpy as np
from sklearn.utils import shuffle
from keras.layers import pooling
from keras.models import Sequential 
from keras.layers import Flatten, Dense, Lambda, Activation
from keras.layers.convolutional import Conv2D
from keras.layers import Dropout
from keras.layers import Cropping2D
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Data pre-processing

In [30]:
lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
#         print(line[0])
        lines.append(line)

# images = []
# measurments = []
# for line in lines:
#     # Create train_X
#     # read in images from center, left and right cameras    
#     image_center = cv2.imread(path + line[0]) # load the center_image
#     image_left = cv2.imread(path + line[1]) # load the left_image
#     image_right = cv2.imread(path + line[2]) # load the right_image
    
#     # append every image to images list
#     images.extend((image_center, image_left, image_right)) 
    
#     # Create train_Y
#     steering_center = float(line[3])# steering measurment
#     # create adjusted steering measurements for the side camera images
#     correction = 0.2 # this is a parameter to tune
#     steering_left = steering_center + correction
#     steering_right = steering_center - correction
    
#     measurments.extend((steering_center, steering_left, steering_right))
    
# images = np.array(images)
# measurments = np.array(measurments)

# # Data augumentation: flip image in order to let it includes both clockwise and counter-clockwise images
# flip_images = []
# measurments_flipped = []
# for image, measurement in zip(images, measurements):
#     flip_images.append(np.fliplr(image))
#     measurments_flipped.append(-measurement)
# flip_images = np.array(flip_images)
# measurments_flipped = np.array(measurments_flipped)

# train_X = np.array(np.concatenate([images, flip_images], axis=0))
# train_Y = np.array(np.concatenate([measurments, measurments_flipped], axis=0))

# # train, validation split
# train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [31]:
def data_generator(samples, batch_size=50):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for start_point in range(0, num_samples, batch_size):
            batch_samples = samples[start_point : start_point + batch_size]
            
            images = []
            angles = []
            for sample in batch_samples:
                name = './data/' + sample[0]
            # Create train_X
            # read in images from center, left and right cameras    
                image_center = cv2.imread(path + line[0]) # load the center_image
                image_left = cv2.imread(path + line[1]) # load the left_image
                image_right = cv2.imread(path + line[2]) # load the right_image
    
            # append every image to images list
                images.extend((image_center, image_left, image_right)) 
    
            # Create train_Y
                steering_center = float(line[3])# steering measurment
            # create adjusted steering measurements for the side camera images
                correction = 0.2 # this is a parameter to tune
                steering_left = steering_center + correction
                steering_right = steering_center - correction
    
                angles.extend((steering_center, steering_left, steering_right))
    
            images = np.array(images)
            angles = np.array(angles)
            
            # Data augumentation: flip image in order to let it includes both clockwise and counter-clockwise images
            images_flipped = []
            angles_flipped = []
            for image, angle in zip(images, angles):
                images_flipped.append(np.fliplr(image))
                angles_flipped.append(-angle)
            images_flipped = np.array(images_flipped)
            angles_flipped = np.array(angles_flipped)

            train_X = np.array(np.concatenate([images, images_flipped], axis=0))
            train_Y = np.array(np.concatenate([angles, angles_flipped], axis=0))
            yield sklearn.utils.shuffle(train_X, train_Y)

In [32]:
# train, validation split
# actually we split the path but not total data, the data will be read by generator
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

# compile and train the model using the generator function
train_generator = data_generator(train_samples, batch_size=32)
validation_generator = data_generator(validation_samples, batch_size=32)

# To build the Network Archtecture

In [34]:
h, w, d = 160, 320, 3
cropping_upper, cropping_bottom = 50, 20

model = Sequential()

model.add(Cropping2D(cropping=((cropping_upper,cropping_bottom), (0,0)), input_shape=(h, w, d)))

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(h-(cropping_upper+cropping_bottom), w, d)))
model.add(BatchNormalization())
                   
model.add(Conv2D(24, (5,5), strides=(2,2), input_shape=(h-(cropping_upper+cropping_bottom), w, d),
                        activation='relu'))
model.add(Dropout(0.5))

model.add(Conv2D(36, (5,5), strides=(2,2), activation='relu'))
# model.add(Dropout(0.5))

model.add(Conv2D(48, (5,5), strides=(2,2), activation='relu'))
# model.add(Dropout(0.5))

model.add(Conv2D(64, (3,3), strides=(1,1), activation='relu'))
# model.add(Dropout(0.5))

model.add(Conv2D(64, (3,3), strides=(1,1), activation='relu'))
# model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(1164))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(50))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(10))
model.add(Activation('relu'))
# model.add(Dropout(0.5))

model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

history_object = model.fit_generator(train_generator, samples_per_epoch=
                                    len(train_samples), validation_data=
                                    validation_generator, nb_val_samples=
                                    len(validation_samples), nb_epoch=2,
                                    verbose=1)

model.save('Behavioral_Cloning_model.h5')

### print the keys contained in the history object 
print(history_object.history.keys())

### plot the training and validation loss for epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

/Users/congcong/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/congcong/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., verbose=1, steps_per_epoch=15742, epochs=2, validation_steps=3936)`


Epoch 1/2


StopIteration: 